In [1]:
import os
import sys
import matplotlib.pylab as plt
import numpy as np
import statistics as stat
%matplotlib notebook

from ztf_hdf5 import Read_LightCurve

import bokeh
import bokeh.plotting
bokeh.plotting.output_notebook()
from bokeh.plotting import figure, show
from bokeh.sampledata.penguins import data
from bokeh.transform import factor_cmap, factor_mark
from bokeh.util.compiler import TypeScript
from bokeh.models import MathML

Loading BokehJS ...

In [2]:
class_faintSN = Read_LightCurve(file_name='Meta_fit_faintSN.hdf5', inputDir='/Users/manon/dataLC')
meta_faintSN = class_faintSN.get_table(path='meta')
meta_faintSN

z,t0,x0,x1,c,mwebv,ra,dec,mwebv_sfd98,idx_orig,path,n_phase_neg,n_phase_pos,n_phase_min,n_phase_max,n_r_band,n_g_band,n_i_band,sel,z_fit,z_err,t0_fit,t0_err,x0_fit,x0_err,x1_fit,x1_err,c_fit,c_err,chisq,ndof,z_t0_cov,z_x0_cov,z_x1_cov,z_c_cov,x0_t0_cov,x0_x1_cov,x0_c_cov,t0_x1_cov,t0_c_cov,x1_c_cov,fitstatus
float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,str7,int64,int64,int64,int64,int64,int64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str8
0.1903221591172223,58183.39502896933,3.72110938951243e-05,-2.0,0.2,0.05932196654659968,333.74427221069857,30.630863721996,0.07090215203250673,643,SN_6344,0,0,0,0,0,0,0,0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,nodata
0.11279388814768768,58491.707977406055,0.00012095254429084725,-2.0,0.2,0.031370818013983975,147.19450207242738,11.430249671676172,0.01932823973902947,644,SN_6345,0,0,0,0,0,0,0,0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,nodata
0.16551842992019505,58305.55234065367,6.382493083140962e-05,-2.0,0.2,0.03189812018604631,27.73646239754698,15.99893152936143,0.043509523472514175,647,SN_6346,0,1,0,0,1,0,0,0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,nodata
0.14321351315348974,58507.413970765585,7.257461855685834e-05,-2.0,0.2,0.0024114689712672645,183.83888643487367,52.77551506309935,0.018640613946046984,648,SN_6347,0,0,0,0,0,0,0,0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,nodata
0.18916293770025866,58229.86774617651,4.256953382799428e-05,-2.0,0.2,0.5617315057300342,105.02736556022626,-0.12642105232500167,0.5558911132033657,651,SN_6348,0,0,0,0,0,0,0,0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,nodata
0.19231683068158711,58351.66945197569,3.6165317834757865e-05,-2.0,0.2,0.042099051472901605,136.19803394926208,19.614555764898476,0.031026096080755163,653,SN_6349,0,0,0,0,0,0,0,0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,nodata
0.1940777401415514,58460.23885326814,3.846189189455665e-05,-2.0,0.2,0.02529560310913301,204.28978233767873,17.890428522888087,0.019817676260582232,654,SN_6350,0,1,0,0,1,0,0,0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,nodata
0.1655258754753213,58384.794713909694,6.218058312954644e-05,-2.0,0.2,0.556683569952051,87.5936971093744,2.881695481091266,0.5573739101870409,655,SN_6351,0,0,0,0,0,0,0,0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,nodata
0.15293984875494337,58446.91746400629,6.620324277518566e-05,-2.0,0.2,0.008430642145982346,200.59948688891268,30.419050516874258,0.011691079348785912,657,SN_6352,0,0,0,0,0,0,0,0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,nodata


In [3]:
from bokeh.plotting import Figure, output_file, show, save, output_notebook

meta_faintSN['chi'] = meta_faintSN['chisq']/meta_faintSN['ndof']

selec = meta_faintSN['sel']==1
msk = meta_faintSN['c_err']!=-1.0
M = selec & msk

data = bokeh.models.ColumnDataSource({
    'redshift':    meta_faintSN["z"][M],
    'err_c':       meta_faintSN["c_err"][M],
    'path':        meta_faintSN["path"][M],
    'chi':         meta_faintSN['chi'][M],
})

fig = bokeh.plotting.figure(
    x_axis_label = '$$z$$',
    y_axis_label = '$$\sigma_{c}$$',
    y_range = (0,0.1),
    plot_width = 800,
    plot_height = 600,
    background_fill_color = 'whitesmoke',
    background_fill_alpha = 0.8
)

fig.add_tools(bokeh.models.HoverTool(tooltips = [('SN path','@path'), ('SN chi','@chi')], mode = 'mouse',))
fig.ray(x=[0],y=[0.04],length=300, angle=0, color='red', legend_label='0.04')
fig.legend.location = "top_left"
fig.legend.click_policy="hide"

fig.scatter(x='redshift', y='err_c', source=data, fill_alpha=0.2, size=10)

bokeh.plotting.show(fig)

In [4]:
from bokeh.plotting import Figure, output_file, show, save, output_notebook, curdoc
from bokeh.layouts import column

selec = meta_faintSN['sel']==1
msk = meta_faintSN['c_err']!=-1.0

MASK = meta_faintSN['n_i_band']==0
MASK2 = meta_faintSN['n_i_band']!=0
M = selec & msk & MASK 
M2 = selec & msk & MASK2
M3 = selec & msk

m1 = meta_faintSN.copy()[M]
m2 = meta_faintSN.copy()[M2]

data = bokeh.models.ColumnDataSource({
    'redshift_rg':    m1["z"],
    'err_c_rg':       m1["c_err"],
    'path_rg':        m1["path"],})
data2 = bokeh.models.ColumnDataSource({
    'redshift_rgi':   m2["z"],
    'err_c_rgi':      m2["c_err"],
    'path_rgi':       m2["path"],})

fig = bokeh.plotting.figure(
    x_axis_label = '$$z$$',
    y_axis_label = '$$\sigma_{c}$$',
    y_range = (0,0.1),
    plot_width = 800,
    plot_height = 600,
    background_fill_color = 'whitesmoke',
    background_fill_alpha = 0.8
)

fig.scatter(x='redshift_rg', y='err_c_rg', source=data, fill_alpha=0.2, size=10, legend_label='r,g band', color='LightSeaGreen')
fig.scatter(x='redshift_rgi', y='err_c_rgi', source=data2, fill_alpha=0.2, size=10, legend_label='r,g,i band', color='FireBrick')

fig.add_tools(bokeh.models.HoverTool(tooltips = [('SN (rg) path ','@path_rg'),('SN (rgi) path ','@path_rgi')], mode = 'mouse',))

fig.ray(x=[0],y=[0.04],length=300, angle=0, color='red', legend_label='0.04')
fig.legend.location = "top_left"
fig.legend.click_policy="hide"

bokeh.plotting.show(fig)